In [1]:
using Revise
using Bilevel

using RigidBodyDynamics

┌ Info: Precompiling Bilevel [9688c538-179f-11e9-3174-495cea6b7f67]
└ @ Base loading.jl:1187


In [3]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), [0.,0.,0.])
normal = FreeVector3D(default_frame(floor), [0.,0.,1.])
floor_obs = Obstacle(floor, point, normal, :xyz, .5)

obstacles = [floor_obs]

1-element Array{Obstacle,1}:
 Obstacle(RigidBody: "floor", Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false)

In [5]:
env = Environment(mechanism, urdf, obstacles)

Environment(Contact[Contact(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., ball, Point3D in "after_floor_to_ball": [0.0, 0.0, 0.0], Obstacle(floor, Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false))])

In [47]:
vs = VarSelector()

VarSelector(Dict{Symbol,UnitRange{Int64}}(), 0)

In [48]:
add_var!(vs, :test, 3)

3

In [50]:
add_var!(vs, :test2, 3)

ArgumentError: ArgumentError: Variable name already exists: test2

In [72]:
add_var!(vs, :none, 0)

ArgumentError: ArgumentError: Variable name 'none' already exists

In [55]:
x = rand(vs.count)

6-element Array{Float64,1}:
 0.6436839941896892
 0.7396478503361856
 0.8049087298045854
 0.716328954764629 
 0.6183191459054316
 0.2071885488224643

In [69]:
0:0 in -1:1

false